In [1]:
import numpy as np

In [2]:
count_table = np.zeros((29,29,29,29,26))
# x is prev
# y is next
# z is frequency of char
# 27th is begin
# 28th is end

In [3]:
def perms(chars,ix):
    n = len(chars)
    
    if ix==n:
        return np.array([])
    
    if chars[ix]==26 or chars[ix]==27:
        list = perms(chars,ix+1)
        list2 = np.empty((0,n-ix))
        for arr in list:
            arr1 = np.append(arr,chars[ix])
            list2 = np.vstack((list2, arr1))
        if(len(list2)==0):
                list2 = np.array([chars[ix]])
        return list2
    
    list = perms(chars,ix+1)
    list2 = np.empty((0,n-ix))
    for arr in list:
        arr1 = np.append(arr,chars[ix])
        list2 = np.vstack((list2, arr1))
        arr2 = np.append(arr,28)
        list2 = np.vstack((list2, arr2))
    if(len(list2)==0):
            list2 = np.array([chars[ix],28])
    return list2

In [4]:
def is_match(perm, word):
    flag = True
    for i in range(4):
      for j in range(i+1,4):
        if word[i]==word[j] and perm[i]!=perm[j]:
          return False
    return True
    

In [5]:
def filter_perm(p,word):
    list2 = np.empty((0,4))
    for i,arr in enumerate(p):
        count = 0
        for j in range(4):
            if arr[j]==28:
                count+=1
        if is_match(arr,word) and count<=1:            
            list2 = np.vstack((list2, arr))
    
    return list2

In [6]:
def rev_perm(p):
    n = len(p)
    for i,arr in enumerate(p):
        p[i] = np.flip(arr)
    
    return p

In [7]:
def encoded_list(word):
    if(len(word)<4): 
        return []
    list_e = np.empty((0,4))
    n = len(word)
    
    for i in range(1,n):
        ch1 = 26
        ch2 = ord(word[i-1]) - ord('a')
        ch3 = ord(word[i]) - ord('a')
        ch4 = 27
        if i!=1:
            ch1 = ord(word[i-2]) - ord('a')
        if i!=n-1:
            ch4 = ord(word[i+1]) - ord('a')
        
        perm = perms(np.array([ch1,ch2,ch3,ch4]),0)
        perm = filter_perm(perm,[ch1,ch2,ch3,ch4])
        perm = rev_perm(perm)
        list_e = np.vstack((list_e, perm))
        
    return list_e

In [8]:
text_file = open("words_250000_train.txt","r")

In [9]:
full_dictionary = text_file.read().splitlines()

In [10]:
for word in full_dictionary:
    
    if(len(word)<4): 
        continue
    
    n = len(word)
    
    for j in range(1,n):
        list_e = np.empty((0,4))
        
        # Make string
        ch1 = 26
        ch2 = ord(word[j-1]) - ord('a')
        ch3 = ord(word[j]) - ord('a')
        ch4 = 27
        if j!=1:
            ch1 = ord(word[j-2]) - ord('a')
        if j!=n-1:
            ch4 = ord(word[j+1]) - ord('a')
            
            
        word1 = [ch1,ch2,ch3,ch4]        
        
        # Get perms
        perm = perms(np.array([ch1,ch2,ch3,ch4]),0)
        perm = filter_perm(perm,word1)
        perm = rev_perm(perm)
        list_e = np.vstack((list_e, perm))
        
        # upd table
        for i in range(len(list_e)):
            if (int)(list_e[i][0])==28:
                count_table[28][(int)(list_e[i][1])][(int)(list_e[i][2])][(int)(list_e[i][3])][word1[0]] += 1
            if (int)(list_e[i][1])==28:
                count_table[(int)(list_e[i][0])][28][(int)(list_e[i][2])][(int)(list_e[i][3])][word1[1]] += 1
            if (int)(list_e[i][2])==28:
                count_table[(int)(list_e[i][0])][(int)(list_e[i][1])][28][(int)(list_e[i][3])][word1[2]] += 1
            if (int)(list_e[i][3])==28:
                count_table[(int)(list_e[i][0])][(int)(list_e[i][1])][(int)(list_e[i][2])][28][word1[3]] += 1

In [ ]:
for i in range(29):
    for j in range(29):
        for k in range(29):
            for m in range(29):
                sum = 0
                for n in range(26):
                    sum+=count_table[i][j][k][m][n]
                if sum!=0:
                    for n in range(26):
                        count_table[i][j][k][m][n]/=sum

In [ ]:
np.save("Count_Table.npy",count_table)